In [1]:
%load_ext autoreload
%autoreload 2

import torch 

from vlstm_nogatematrices import vlstm_fw_prepare_gate_preacts, vlstm_fw_nogatematrices_nostabilization, vlstm_fwbw_nogatematrices_nostabilization
from vlstm_full import vlstm_fw_torch



# vLSTM forward backward (NOGATES notstabilized) implementation

In [2]:
DTYPE = torch.float32 
DEVICE = torch.device("cuda:0")

In [3]:
B = 1
S = 5
NH = 1
DH = 6
EPS = 0.0

In [4]:
igate_preacts = torch.randn((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
fgate_preacts = torch.randn((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
temp_Ctilde = torch.zeros((B, NH, S, S), dtype=DTYPE, device=DEVICE)
temp_D = torch.zeros((B, NH, S, S), dtype=DTYPE, device=DEVICE)
temp_QK = torch.zeros((B, NH, S, S), dtype=DTYPE, device=DEVICE)
temp_N = torch.zeros((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
temp_B = torch.zeros((B, NH, S, 1), dtype=DTYPE, device=DEVICE)

In [5]:
fgate_mat, igate_mat = vlstm_fw_prepare_gate_preacts(igate_preacts, fgate_preacts)
igate_mat.shape, fgate_mat.shape

(torch.Size([1, 1, 5, 5]), torch.Size([1, 1, 5, 5]))

In [6]:
fgate_mat

tensor([[[[ 0.0000,    -inf,    -inf,    -inf,    -inf],
          [-0.4513,  0.0000,    -inf,    -inf,    -inf],
          [-0.8358, -0.3845,  0.0000,    -inf,    -inf],
          [-1.4699, -1.0186, -0.6341,  0.0000,    -inf],
          [-1.6417, -1.1903, -0.8058, -0.1717,  0.0000]]]], device='cuda:0')

In [7]:
igate_mat

tensor([[[[-0.5616,    -inf,    -inf,    -inf,    -inf],
          [-0.5616, -1.9604,    -inf,    -inf,    -inf],
          [-0.5616, -1.9604,  0.6901,    -inf,    -inf],
          [-0.5616, -1.9604,  0.6901, -1.7415,    -inf],
          [-0.5616, -1.9604,  0.6901, -1.7415,  0.6842]]]], device='cuda:0')

In [8]:
qs = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
ks = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
vs = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
vs.shape

torch.Size([1, 1, 5, 6])

## Backward NOT stabilized without input & forget gate

### Torch Autograd

In [9]:
fgate_mat_pt = fgate_mat.clone().detach().requires_grad_(True)
igate_mat_pt = igate_mat.clone().detach().requires_grad_(True)
qs_pt = qs.clone().detach().requires_grad_(True)
ks_pt = ks.clone().detach().requires_grad_(True)
vs_pt = vs.clone().detach().requires_grad_(True)
temp_Ctilde_pt = temp_Ctilde.clone().detach().requires_grad_(True)
temp_D_pt = temp_D.clone().detach().requires_grad_(True)
temp_QK_pt = temp_QK.clone().detach().requires_grad_(True)
temp_N_pt = temp_N.clone().detach().requires_grad_(True)
temp_B_pt = temp_B.clone().detach().requires_grad_(True)

In [10]:
igate_mat_pt, fgate_mat_pt

(tensor([[[[-0.5616,    -inf,    -inf,    -inf,    -inf],
           [-0.5616, -1.9604,    -inf,    -inf,    -inf],
           [-0.5616, -1.9604,  0.6901,    -inf,    -inf],
           [-0.5616, -1.9604,  0.6901, -1.7415,    -inf],
           [-0.5616, -1.9604,  0.6901, -1.7415,  0.6842]]]], device='cuda:0',
        requires_grad=True),
 tensor([[[[ 0.0000,    -inf,    -inf,    -inf,    -inf],
           [-0.4513,  0.0000,    -inf,    -inf,    -inf],
           [-0.8358, -0.3845,  0.0000,    -inf,    -inf],
           [-1.4699, -1.0186, -0.6341,  0.0000,    -inf],
           [-1.6417, -1.1903, -0.8058, -0.1717,  0.0000]]]], device='cuda:0',
        requires_grad=True))

In [11]:
retr_val_pt = vlstm_fw_nogatematrices_nostabilization(
    qs_pt, ks_pt, vs_pt, igate_mat_pt, fgate_mat_pt, 
    temp_Ctilde_pt,
    temp_D_pt, 
    temp_QK_pt,
    temp_N_pt,
    temp_B_pt,
    eps=EPS
)
retr_val_pt.shape

torch.Size([1, 1, 5, 6])

In [12]:
retr_val_pt.sum().backward()

In [13]:
temp_Ctilde_pt.grad

tensor([[[[ 0.8814,  0.9833, -1.2489, -0.6811, -1.2131],
          [ 0.8814,  0.9833, -1.2489, -0.6811, -1.2131],
          [ 0.8814,  0.9833, -1.2489, -0.6811, -1.2131],
          [ 0.8814,  0.9833, -1.2489, -0.6811, -1.2131],
          [ 1.6148,  1.6998, -0.1633,  0.3106, -0.1335]]]], device='cuda:0')

In [14]:
qs_pt.grad

tensor([[[[ 0.1050, -0.0625, -0.0948,  0.1407, -0.0540,  0.1197],
          [-0.0367, -0.0734, -0.0808,  0.0606, -0.1055,  0.1174],
          [ 0.1340, -0.3969, -1.3883, -0.3274,  0.1434,  1.3614],
          [ 0.0710, -0.1397, -0.7741, -0.2373, -0.0184,  0.6738],
          [ 0.0300, -0.0314, -0.0784, -0.0532, -0.0176,  0.0239]]]],
       device='cuda:0')

In [15]:
ks_pt.grad

tensor([[[[-0.1706,  0.1444,  0.2425,  0.0425, -0.5700, -0.4328],
          [-0.0860, -0.0919,  0.0851, -0.0087, -0.0898, -0.0742],
          [ 1.6149, -0.7717,  0.2684, -0.0029,  1.0820,  0.5930],
          [ 0.0223, -0.0520,  0.0273, -0.0127,  0.0155,  0.0521],
          [ 0.0602,  0.0394, -0.0290,  0.1112, -0.0174,  0.1158]]]],
       device='cuda:0')

In [16]:
vs_pt.grad

tensor([[[[-0.2841, -0.2841, -0.2841, -0.2841, -0.2841, -0.2841],
          [ 0.2384,  0.2384,  0.2384,  0.2384,  0.2384,  0.2384],
          [ 1.2360,  1.2360,  1.2360,  1.2360,  1.2360,  1.2360],
          [-0.3435, -0.3435, -0.3435, -0.3435, -0.3435, -0.3435],
          [-1.2605, -1.2605, -1.2605, -1.2605, -1.2605, -1.2605]]]],
       device='cuda:0')

In [17]:
fgate_mat_pt.grad

tensor([[[[-0.1227, -0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0063,  0.0926, -0.0000, -0.0000, -0.0000],
          [-0.0171,  0.1287, -0.1539,  0.0000, -0.0000],
          [-0.0615,  0.0033, -0.9129,  0.1873,  0.0000],
          [-0.1217,  0.0203, -0.0747, -0.0255,  0.2016]]]], device='cuda:0')

In [18]:
igate_mat_pt.grad

tensor([[[[-0.1227, -0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0063,  0.0926, -0.0000, -0.0000, -0.0000],
          [-0.0171,  0.1287, -0.1539,  0.0000, -0.0000],
          [-0.0615,  0.0033, -0.9129,  0.1873,  0.0000],
          [-0.1217,  0.0203, -0.0747, -0.0255,  0.2016]]]], device='cuda:0')

### Own backward

In [19]:
fgate_mat_obw = fgate_mat.clone().detach().requires_grad_(True)
igate_mat_obw = igate_mat.clone().detach().requires_grad_(True)
qs_obw = qs.clone().detach().requires_grad_(True)
ks_obw = ks.clone().detach().requires_grad_(True)
vs_obw = vs.clone().detach().requires_grad_(True)
temp_Ctilde_obw = temp_Ctilde.clone().detach().requires_grad_(True)
temp_D_obw = temp_D.clone().detach().requires_grad_(True)
temp_QK_obw = temp_QK.clone().detach().requires_grad_(True)
temp_N_obw = temp_N.clone().detach().requires_grad_(True)
temp_B_obw = temp_B.clone().detach().requires_grad_(True)

In [20]:
retr_val_obw = vlstm_fwbw_nogatematrices_nostabilization(
    qs_obw, ks_obw, vs_obw, igate_mat_obw, fgate_mat_obw, 
    temp_Ctilde_obw,
    temp_D_obw, 
    temp_QK_obw,
    temp_N_obw,
    temp_B_obw,
    eps=EPS
)
retr_val_obw.shape

torch.Size([1, 1, 5, 6])

In [21]:
retr_val_obw.sum().backward()

In [22]:
temp_N_obw.grad - temp_N_pt.grad

tensor([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.]]]], device='cuda:0')

In [23]:
temp_B_obw.grad - temp_B_pt.grad

tensor([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.]]]], device='cuda:0')

In [24]:
temp_QK_obw.grad - temp_QK_pt.grad

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], device='cuda:0')

In [25]:
temp_Ctilde_obw.grad - temp_Ctilde_pt.grad

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], device='cuda:0')

In [26]:
qs_obw.grad-qs_pt.grad

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [27]:
ks_obw.grad-ks_pt.grad

tensor([[[[ 0.0000e+00,  0.0000e+00,  1.4901e-08,  3.7253e-09,  0.0000e+00,
            2.9802e-08],
          [ 0.0000e+00,  7.4506e-09,  7.4506e-09, -9.3132e-10,  0.0000e+00,
            7.4506e-09],
          [ 0.0000e+00,  5.9605e-08, -2.9802e-08,  2.0955e-09, -1.1921e-07,
           -5.9605e-08],
          [ 0.0000e+00, -3.7253e-09, -3.7253e-09, -9.3132e-10, -9.3132e-10,
            0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  1.8626e-09, -7.4506e-09,  1.8626e-09,
           -7.4506e-09]]]], device='cuda:0')

In [28]:
vs_obw.grad-vs_pt.grad

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [29]:
fgate_mat_obw.grad-fgate_mat_pt.grad

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], device='cuda:0')

### Do gradients match? 

In [30]:
qs_pt.grad - qs_obw.grad

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [31]:
atol = 1e-6
rtol = 1e-6
print(f"Forward match: {torch.allclose(retr_val_pt, retr_val_obw)}")
print(f"qs match: {torch.allclose(qs_pt.grad, qs_obw.grad, atol=atol, rtol=rtol)}")
print(f"ks match: {torch.allclose(ks_pt.grad, ks_obw.grad, atol=atol, rtol=rtol)}")
print(f"vs match: {torch.allclose(vs_pt.grad, vs_obw.grad, atol=atol, rtol=rtol)}")
print(f"fgate_mat match: {torch.allclose(fgate_mat_pt.grad, fgate_mat_obw.grad, atol=atol, rtol=rtol)}")
print(f"igate_mat match: {torch.allclose(igate_mat_pt.grad, igate_mat_obw.grad, atol=atol, rtol=rtol)}")

Forward match: True
qs match: True
ks match: True
vs match: True
fgate_mat match: True
igate_mat match: True


## Forward without input & forget gate

In [32]:
retr_vals = vlstm_fw_nogatematrices_nostabilization(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igate_mat,
    fgate_preact=fgate_mat,
    temp_Ctilde=temp_Ctilde,
    temp_D=temp_D,
    temp_QK=temp_QK,
    temp_N=temp_N,
    temp_B=temp_B,
    eps=EPS,
)
retr_vals.shape

torch.Size([1, 1, 5, 6])

In [33]:
retr_vals_fwbw = vlstm_fwbw_nogatematrices_nostabilization(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igate_mat,
    fgate_preact=fgate_mat,
    temp_Ctilde=temp_Ctilde,
    temp_D=temp_D,
    temp_QK=temp_QK,
    temp_N=temp_N,
    temp_B=temp_B,
    eps=EPS,
)
retr_vals_fwbw.shape

torch.Size([1, 1, 5, 6])

### Check if it equals the full version:

In [34]:
# check if equals the full version
retr_vals_full = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igate_preacts, fgate_preact=fgate_preacts)
retr_vals_full.shape

torch.Size([1, 1, 5, 6])

In [35]:
# The implementations match!!!
retr_vals - retr_vals_full

tensor([[[[-2.0955e-09,  1.8626e-08,  2.6077e-08, -8.9407e-08, -2.3283e-09,
           -1.8626e-08],
          [ 7.4506e-09,  2.9802e-08,  1.8626e-08, -2.9802e-08,  7.4506e-09,
            3.7253e-09],
          [-7.8231e-08,  4.0233e-07,  5.0664e-07, -7.1526e-07, -5.5879e-08,
           -1.5646e-07],
          [-7.1526e-07,  8.9407e-07,  4.7684e-07, -4.1723e-07, -8.9407e-07,
           -1.1921e-07],
          [-1.4305e-06,  5.4836e-06,  2.8610e-06, -1.6093e-06,  7.1526e-07,
           -4.2915e-06]]]], device='cuda:0')

In [36]:
retr_vals_fwbw - retr_vals_full

tensor([[[[-2.0955e-09,  1.8626e-08,  2.6077e-08, -8.9407e-08, -2.3283e-09,
           -1.8626e-08],
          [ 7.4506e-09,  2.9802e-08,  1.8626e-08, -2.9802e-08,  7.4506e-09,
            3.7253e-09],
          [-7.8231e-08,  4.0233e-07,  5.0664e-07, -7.1526e-07, -5.5879e-08,
           -1.5646e-07],
          [-7.1526e-07,  8.9407e-07,  4.7684e-07, -4.1723e-07, -8.9407e-07,
           -1.1921e-07],
          [-1.4305e-06,  5.4836e-06,  2.8610e-06, -1.6093e-06,  7.1526e-07,
           -4.2915e-06]]]], device='cuda:0')